# ELEC 474 Lab2: K-Means Clustering

Matthieu Roux - 20013052

For Lab 2 we will be implementing a k-means clustering algorithm. We will perform color quantization on an image using k-means with random center(s) initialization

## Imports

In [4]:
import cv2
import numpy as np
import copy

First randomly select points within the image bounds based on your selected number of
clusters, 𝒌. You should create a track bar to select and experiment with different values of 𝒌.
Your randomly selected cluster centers, { ci}, i = 1..k, should include color channel intensities, as
they will be used to calculate your means.

In [ ]:
# code for step 1 here

In [ ]:
Partition your image into Voronoi Regions based on the color values of each pixel. The Voronoi
Region around each cluster center ci includes all pixels that are closest to that cluster center,
according to some distance metric. In this case, as we are only interested in the color values of
the pixels, you should use a distance metric based solely on the color channels to determine
which pixels belong to which cluster.


In [ ]:
# code for step 2

In [ ]:
Find the mean color value mi for each Voronoi Region.

a) If for each region, the mean values are very close to the cluster centers (i.e. mi ci for all i =
1..k ), then the algorithm has converged. Recolor all pixels in each region to their mean color
values, and STOP.
b) Else, if any of the mean values differ significantly from their cluster centers (i.e. mi  ci for any
1..k), then update the cluster centers to equal the mean values (i.e. set ci = mi) and repeat
Steps 2 and 3.

In [ ]:
# code for step 3
